In [63]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.{DataFrame}
import org.apache.spark.sql.functions
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline,PipelineModel}
import org.apache.spark.ml.feature.Imputer

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.Imputer


In [2]:
val spark = SparkSession
    .builder()
    .appName("Utils")
    .master("local[2]")
    .getOrCreate()

val sc = spark.sparkContext

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7909e424
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@12b9fdb8


In [32]:
val df = spark.read.format("csv").option("header",true).load("./data/train.csv")

df: org.apache.spark.sql.DataFrame = [PassengerId: string, Survived: string ... 10 more fields]


In [33]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [34]:
df.filter(col("Pclass")===1).groupBy("Survived","Sex").agg(functions.count("PassengerId").alias("passengers_cnt"),
                                                            functions.round(functions.mean("Age")).alias("mean_age")).show()

+--------+------+--------------+--------+
|Survived|   Sex|passengers_cnt|mean_age|
+--------+------+--------------+--------+
|       1|  male|            45|    36.0|
|       1|female|            91|    35.0|
|       0|female|             3|    26.0|
|       0|  male|            77|    45.0|
+--------+------+--------------+--------+



In [39]:
val udf = functions.udf { (x:String) => if(x=="male") 1 else 0}

udf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))


In [40]:
val new_df = df.withColumn("sex",udf(df("Sex")))

new_df: org.apache.spark.sql.DataFrame = [PassengerId: string, Survived: string ... 10 more fields]


In [41]:
new_df.show()

+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  1|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|  0|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|  0|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|  0|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  1|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  1|null|    0|    0|          330877| 8.4583| null|  

In [49]:
var features = new_df.columns.filterNot(_.contains("Name")).filterNot(_.contains("PassengerId"))

features: Array[String] = Array(Survived, Pclass, sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked)


In [56]:
val encoder = Array("Ticket","Cabin","Embarked").flatMap{name=>
    val indexer = new StringIndexer().setInputCol(name).setOutputCol(name+"_encode").setHandleInvalid("keep")
    Array(indexer)
}.toArray

encoder: Array[org.apache.spark.ml.feature.StringIndexer] = Array(strIdx_127d34ff522f, strIdx_3e90cad96d88, strIdx_71fe82e0deae)


In [57]:
val pipline = new Pipeline().setStages(encoder)
val indexModel = pipline.fit(new_df)

pipline: org.apache.spark.ml.Pipeline = pipeline_3c2fc629bd1f
indexModel: org.apache.spark.ml.PipelineModel = pipeline_3c2fc629bd1f


In [58]:
val featureDF = indexModel.transform(new_df)

featureDF: org.apache.spark.sql.DataFrame = [PassengerId: string, Survived: string ... 13 more fields]


In [62]:
val featureDF2 = featureDF.drop(Array("Ticket","Cabin","Embarked","Name","PassengerId"):_*)
featureDF2.show()

+--------+------+---+----+-----+-----+-------+-------------+------------+---------------+
|Survived|Pclass|sex| Age|SibSp|Parch|   Fare|Ticket_encode|Cabin_encode|Embarked_encode|
+--------+------+---+----+-----+-----+-------+-------------+------------+---------------+
|       0|     3|  1|  22|    1|    0|   7.25|        257.0|       147.0|            0.0|
|       1|     1|  0|  38|    1|    0|71.2833|        608.0|        87.0|            1.0|
|       1|     3|  0|  26|    0|    0|  7.925|        292.0|       147.0|            0.0|
|       1|     1|  0|  35|    1|    0|   53.1|         46.0|        42.0|            0.0|
|       0|     3|  1|  35|    0|    0|   8.05|        425.0|       147.0|            0.0|
|       0|     3|  1|null|    0|    0| 8.4583|        269.0|       147.0|            2.0|
|       0|     1|  1|  54|    0|    0|51.8625|        438.0|       102.0|            0.0|
|       0|     3|  1|   2|    3|    1| 21.075|         12.0|       147.0|            0.0|
|       1|

featureDF2: org.apache.spark.sql.DataFrame = [Survived: string, Pclass: string ... 8 more fields]


In [80]:
val castDF = featureDF2.select(featureDF2.columns.map{
        colname=>col(colname).cast("double").alias(colname)
    
}:_*)

castDF: org.apache.spark.sql.DataFrame = [Survived: double, Pclass: double ... 8 more fields]


In [81]:
castDF.printSchema()

root
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- sex: double (nullable = false)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Ticket_encode: double (nullable = false)
 |-- Cabin_encode: double (nullable = false)
 |-- Embarked_encode: double (nullable = false)



In [82]:
val imputer = new Imputer()
.setInputCols(featureDF2.columns.takeRight(9))
.setOutputCols(featureDF2.columns.takeRight(9).map{ name=> name+"_inputed"})
.fit(castDF)

imputer: org.apache.spark.ml.feature.ImputerModel = imputer_838bf992fd26


In [86]:
val imputedDF = imputer.transform(castDF).drop(imputer.getInputCols:_*)
imputedDF.show()

+--------+-----------------+-------------+---------------------+------------+-------------+-----------------------+-----------+--------------+--------------------+
|Survived|      Age_inputed|SibSp_inputed|Ticket_encode_inputed|Fare_inputed|Parch_inputed|Embarked_encode_inputed|sex_inputed|Pclass_inputed|Cabin_encode_inputed|
+--------+-----------------+-------------+---------------------+------------+-------------+-----------------------+-----------+--------------+--------------------+
|     0.0|             22.0|          1.0|                257.0|        7.25|          0.0|                    0.0|        1.0|           3.0|               147.0|
|     1.0|             38.0|          1.0|                608.0|     71.2833|          0.0|                    1.0|        0.0|           1.0|                87.0|
|     1.0|             26.0|          0.0|                292.0|       7.925|          0.0|                    0.0|        0.0|           3.0|               147.0|
|     1.0|      

imputedDF: org.apache.spark.sql.DataFrame = [Survived: double, Age_inputed: double ... 8 more fields]
